In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
import zipfile

# Mount Google Drive
drive.mount('/content/drive')

# Unzip the dataset
zip_path = '/content/drive/MyDrive/archive.zip'  # Adjust path if needed
extract_path = '/content/drive/MyDrive/fer2013'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
train_base_dir = "/content/drive/MyDrive/fer2013/train"
test_base_dir = "/content/drive/MyDrive/fer2013/test"

print("Training categories:", os.listdir(train_base_dir))
print("Testing categories:", os.listdir(test_base_dir))

# Store available categories
category_list = os.listdir(train_base_dir)


In [ ]:
fig = plt.figure(figsize=(10, 10))
i = 1
for category in category_list:
    category_path = os.path.join(train_base_dir, category)
    img_name = os.listdir(category_path)[0]
    img_path = os.path.join(category_path, img_name)
    img_bgr = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    plt.subplot(1, 7, i)
    plt.imshow(img_rgb)
    plt.title(category)
    plt.axis('off')
    i += 1
plt.show()


In [ ]:
train_data_gen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255
)
test_data_gen = ImageDataGenerator(rescale=1./255)

train_data = train_data_gen.flow_from_directory(
    train_base_dir,
    target_size=(224, 224),
    class_mode='sparse',
    batch_size=32
)

test_data = test_data_gen.flow_from_directory(
    test_base_dir,
    target_size=(224, 224),
    class_mode='sparse',
    batch_size=32
)


In [ ]:
class_indices = train_data.class_indices
category_mapping = {v: k for k, v in class_indices.items()}
print(category_mapping)


In [ ]:
from tensorflow.keras.applications import MobileNetV3Large

model = MobileNetV3Large()
for layer in model.layers:
    layer.trainable = False

output = model.layers[-1].output
output = BatchNormalization()(output)
output = Dense(128)(output)
output = BatchNormalization()(output)
output = Dense(7, activation="softmax")(output)

new_model = tf.keras.Model(inputs=model.input, outputs=output)
new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

new_model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlystop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=3)

history = new_model.fit(
    train_data,
    epochs=10,
    validation_data=test_data,
    callbacks=[earlystop, reduce_lr]
)


Epoch 1/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 1776s 2s/step - accuracy: 0.1682 - loss: 2.0433 - val_accuracy: 0.2473 - val_loss: 1.8463 - learning_rate: 1.0000e-04
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 1764s 2s/step - accuracy: 0.2234 - loss: 1.8596 - val_accuracy: 0.2480 - val_loss: 1.8157 - learning_rate: 1.0000e-04
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 1627s 2s/step - accuracy: 0.2320 - loss: 1.8338 - val_accuracy: 0.2488 - val_loss: 1.8058 - learning_rate: 1.0000e-04
Epoch 4/10
151/898 ━━━━━━━━━━━━━━━━━━━━ 18:58 2s/step - accuracy: 0.2402 - loss: 1.8166

In [ ]:
# Save the model
new_model.save('/content/drive/MyDrive/my_model.h5')

# Save category mapping
import pickle
with open('/content/drive/MyDrive/category_map.pkl', 'wb') as f:
    pickle.dump(category_mapping, f)


In [ ]:
new_model.save('/content/drive/MyDrive/my_model.keras')


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/my_model.keras')


In [ ]:
with open('/content/drive/MyDrive/category_map.pkl', 'rb') as f:
    category_mapping = pickle.load(f)
print("Loaded category mapping:", category_mapping)

In [ ]:
import tensorflow as tf
import pickle

# Load the model
model = tf.keras.models.load_model('/content/drive/MyDrive/my_model.keras')

# Load category mapping
with open('/content/drive/MyDrive/category_map.pkl', 'rb') as f:
    category_mapping = pickle.load(f)

print("Loaded category mapping:", category_mapping)
